In [1]:
import shapefile as sf
import numpy as np
myfile = sf.Reader("/g/data1a/ge3/AEM_Model/3D_AEM_model_V3.shp")

In [2]:
myshaperecord = myfile.shapeRecords()

In [3]:
#list all the attribute of a record
myfile.fields

[('DeletionFlag', 'C', 1, 0),
 ['Id', 'N', 10, 0],
 ['unique_id', 'C', 50, 0],
 ['ref_0001', 'F', 13, 11],
 ['ref_0001vs', 'F', 13, 11],
 ['ref_0010', 'F', 13, 11],
 ['ref_0100', 'F', 13, 11],
 ['ref_0100vs', 'F', 13, 11],
 ['thick', 'F', 13, 11],
 ['wii_clip_1', 'F', 13, 11],
 ['ceno_clip_', 'F', 13, 11],
 ['X', 'F', 19, 11],
 ['Y', 'F', 19, 11],
 ['Z', 'F', 19, 11],
 ['class', 'N', 10, 0],
 ['age', 'N', 10, 0]]

In [19]:
# an example record
# note that DeletionFlag was not included
print [#myshaperecord[0].record, 
       myshaperecord[1].record,
       #myshaperecord[2].record
        ]

[[0, 160287, 0.000862, 0.0019, 0.00114, 0.00176, 0.000862, 4.0, 3.37731, 0.00566225, 347931.8, 7339632.5, 581.58, 0, 1]]


In [130]:
#save unique-id, AEM-values, class-id into a txt file

myrecord=(x.record for x in myshaperecord if x.record[-2]>0)

from itertools import cycle, compress

criteria = cycle([False]+[True]*9+[False]*2+[True]*3)

with open('AEM.txt','w') as fw:
    for r in myrecord:
        if r[1]!='Surface':         
            r[1]=int(r[1])         
            np.savetxt(fw, np.array(list(compress(r,criteria))).reshape(1,-1), fmt='%s')

In [31]:
#count the number of distinct unique-id 
#each unique-id corresponds to a drill hole
#surface counts for one

len(set(x.record[1] for x in myshaperecord))
 

42257

In [32]:
min(x.record[1] for x in myshaperecord)

160287

In [33]:
max(x.record[1] for x in myshaperecord if x.record[1]!='Surface')

202542

In [34]:
202542-160287

42255

In [35]:
#steps above shows, the unique-id are continuous between 160287 and 202542
#let us take those >0 and <='200000' as training, and the other as testing
#those =0 are surfaces with non-usable AEM

In [228]:
#count the occurrance of each class in training data

class_count=np.zeros(10)
total_count=0
y=(x.record for x in myshaperecord if x.record[1]<=175000 and x.record[-2]>0)
for x in y:
    class_count[x[13]]=class_count[x[13]]+1
    total_count=total_count+1
print class_count 
print total_count

[  0. 456.  77.  99.  23.   0.   0. 116. 219.  92.]
1082


In [132]:
#count the occurrance of each class in testing data

class_count = np.zeros(10)
total_count = 0
y=(x.record for x in myshaperecord if x.record[1]>175000 and x.record[-2]>0 and x.record[1]!='Surface')

for x in y:
    class_count[x[13]] = class_count[x[13]]+1
    total_count = total_count+1
print class_count 
print total_count

[ 0. 94. 41. 70. 10.  0.  0. 90. 83. 76.]
464


In [133]:
#construct kd tree using the testing data


import scipy as sp
import scipy.spatial
import itertools as it


fr = open('AEM.txt')

lines = np.array([map(float, line.split()) for line in fr])


#extract label, drop label and unique-id
label = lines[:,-2]
lines = np.delete(lines,0,1)
lines = np.delete(lines,-2,1)

#whitening transform
lines_whitened = (lines-lines.min(axis=0))/(lines.max(axis=0)-lines.min(axis=0))

In [135]:
data_index=scipy.spatial.KDTree(lines_whitened)

In [134]:
lines_whitened.shape

(1043, 10)

In [148]:
lines_whitened[26:28]

array([[0.00314949, 0.00743494, 0.00355789, 0.00435484, 0.00314949,
        0.81352459, 0.05343313, 0.        , 0.41118541, 0.        ],
       [0.00320308, 0.00594697, 0.00354698, 0.00470968, 0.00320308,
        0.9023845 , 0.05135549, 0.        , 0.37235569, 0.        ]])

In [142]:
label[26:28]

array([8., 9.])

In [156]:
prediction=data_index.query(lines_whitened,2)

In [161]:
prediction[1][:,1]

array([   1,    0,    1, ..., 1005, 1042, 1041])

In [162]:
prediction[1][:,0]

array([   0,    1,    2, ..., 1040, 1041, 1042])

In [167]:
label_predicted = label[prediction[1][:,1]]

In [178]:
label_predicted[200:210]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [177]:
label[200:210]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [174]:
sum(map(bool,label-label_predicted))

87

In [175]:
87.0/1043

0.08341323106423777

In [193]:
class_count = np.zeros(10)
total_count = 0

for x in label:
    x=int(x)
    class_count[x] = class_count[x]+1
    total_count = total_count+1
print class_count 
print total_count

[  0. 453.  73.  92.  17.   0.   0. 101. 215.  92.]
1043


In [194]:
class_count = np.zeros(10)
total_count = 0

for x in label_predicted:
    x=int(x)
    class_count[x] = class_count[x]+1
    total_count = total_count+1
print class_count 
print total_count

[  0. 444.  82.  90.  18.   0.   0. 105. 213.  91.]
1043


In [221]:
confusion = np.zeros([10,10])

for x in zip(label_predicted,label):
    confusion[tuple(map(int,x))]=confusion[tuple(map(int,x))]+1



In [222]:
confusion[]

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0., 439.,   3.,   1.,   1.,   0.,   0.,   0.,   0.,   0.],
       [  0.,  12.,  58.,   6.,   1.,   0.,   0.,   5.,   0.,   0.],
       [  0.,   1.,   7.,  73.,   0.,   0.,   0.,   9.,   0.,   0.],
       [  0.,   1.,   1.,   1.,  12.,   0.,   0.,   2.,   1.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   4.,  11.,   3.,   0.,   0.,  83.,   4.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   2., 205.,   6.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   5.,  86.]])

In [224]:
y=0
for x in zip(label_predicted,label):
    if x[0]==1 and x[1]==2:
        y=y+1
print y

3


In [231]:
confusion.sum(axis=0)

array([  0., 453.,  73.,  92.,  17.,   0.,   0., 101., 215.,  92.])

In [230]:
confusion.sum(axis=1)

array([  0., 444.,  82.,  90.,  18.,   0.,   0., 105., 213.,  91.])

In [220]:
conf[tuple(map(int,confusion[1]))]

0.0

In [2]:
#construct kd tree using the testing data

import numpy as np
import scipy as sp
import scipy.spatial
import itertools as it


fr = open('AEM.txt')

lines = np.array([map(float, line.split()) for line in fr])

In [7]:
c=0
for x in lines:
    if x[-1] ==1 :
        c=c+1
print c

345


In [9]:
lines[142:144]

array([[1.61608e+05, 3.39000e-02, 1.83000e-02, 3.04000e-02, 2.69000e-02,
        3.39000e-02, 3.25600e+01, 2.64657e+00, 2.10540e-02, 3.50350e+02,
        8.00000e+00, 1.00000e+00],
       [1.61608e+05, 1.33000e-02, 1.39000e-02, 1.56000e-02, 2.63000e-02,
        1.33000e-02, 3.58200e+01, 2.62010e+00, 2.10540e-02, 3.16160e+02,
        9.00000e+00, 1.00000e+00]])

In [217]:
type(map(int ,x))

list

In [233]:
prediction[1]

array([[   0,    1],
       [   1,    0],
       [   2,    1],
       ...,
       [1040, 1005],
       [1041, 1042],
       [1042, 1041]])

In [234]:
fr = open('AEM.txt')

lines = np.array([map(float, line.split()) for line in fr])


#extract label, drop label and unique-id
label = lines[:,-2]
lines = np.delete(lines,0,1)
lines = np.delete(lines,-2,1)

#whitening transform
lines_whitened = (lines-lines.min(axis=0))/(lines.max(axis=0)-lines.min(axis=0))

In [244]:
data_index1=scipy.spatial.KDTree(lines_whitened[0:500])
prediction=data_index1.query(lines_whitened,2)
label_predicted = label[prediction[1][:,0]]

In [245]:
sum(map(bool,label-label_predicted)[500:])

176

In [241]:
60.0/543

0.11049723756906077

In [240]:
len(label[500:])

543

In [387]:
def run(train,test):
    train_label = train[:,-2]
    train_lines = np.delete(train,0,1)
    train_lines = np.delete(train_lines,-2,1)
    
    test_label = test[:,-2]
    test_lines = np.delete(test,0,1)
    test_lines = np.delete(test_lines,-2,1)    

    train_whitened = (train_lines-train_lines.min(axis=0))/(train_lines.max(axis=0)-train_lines.min(axis=0))
    test_whitened = (test_lines-train_lines.min(axis=0))/(train_lines.max(axis=0)-train_lines.min(axis=0))
    
    data_index = scipy.spatial.KDTree(train_whitened)
    prediction = data_index.query(test_whitened,1)
    label_predicted = train_label[prediction[1]]
    
    return float(sum(map(bool,test_label-label_predicted)))/len(test_label)
    

def cross_fold(fid):
    fr = open('AEM.txt')
    lines = np.array([map(float, line.split()) for line in fr])
    folds = list(set(lines[:,0]))
    
    train = np.array([x for x in lines if x[0]!=folds[fid]])
    test = np.array([x for x in lines if x[0]==folds[fid]])
    return run(train,test)


In [388]:
error=[cross_fold(x) for x in range(51)]

In [389]:
np.mean(error)

0.21175788054013026

In [393]:
error

[0.6666666666666666,
 0.3333333333333333,
 0.5,
 0.14814814814814814,
 0.4666666666666667,
 0.5,
 0.26666666666666666,
 0.3,
 0.03333333333333333,
 0.0,
 0.0,
 0.4,
 0.1,
 0.13333333333333333,
 0.0,
 0.1,
 0.06666666666666667,
 0.03333333333333333,
 0.06666666666666667,
 0.3157894736842105,
 0.3333333333333333,
 0.4,
 0.2,
 0.25,
 0.0,
 0.06666666666666667,
 0.0,
 0.2,
 0.23333333333333334,
 0.0,
 0.2,
 0.16666666666666666,
 0.0,
 0.16666666666666666,
 0.0,
 0.1,
 1.0,
 0.5333333333333333,
 0.06666666666666667,
 0.8571428571428571,
 0.2,
 0.14814814814814814,
 0.2,
 0.08333333333333333,
 0.3,
 0.17857142857142858,
 0.18518518518518517,
 0.0,
 0.13333333333333333,
 0.16666666666666666,
 0.0]

In [396]:
np.mean([x for x in error if x<0.5])

0.14983353445341752

In [371]:
len(set(lines[:,0]))

51

In [336]:
class_count = np.zeros(10)
total_count = 0

for x in label[150:]:
    x=int(x)
    class_count[x] = class_count[x]+1
    total_count = total_count+1
print class_count 
print total_count

[  0. 453.  73.  92.  15.   0.   0. 101.  83.  76.]
893


In [386]:

train[:,-2]

TypeError: list indices must be integers, not tuple

In [400]:
myrecord=[x.record[10:13] for x in myshaperecord]

In [401]:
len(myrecord)

1309936

In [404]:
myrecord=np.array(myrecord)

In [405]:
myrecord[0:100,0]

array([347931.8, 347931.8, 347931.8, 347931.8, 347931.8, 347931.8,
       347931.8, 347931.8, 347931.8, 347931.8, 347931.8, 347931.8,
       347931.8, 347931.8, 347931.8, 347931.8, 347931.8, 347931.8,
       347931.8, 347931.8, 347931.8, 347931.8, 347931.8, 347931.8,
       347931.8, 347931.8, 347931.8, 347931.8, 347931.8, 347931.8,
       347931.8, 347942.7, 347942.7, 347942.7, 347942.7, 347942.7,
       347942.7, 347942.7, 347942.7, 347942.7, 347942.7, 347942.7,
       347942.7, 347942.7, 347942.7, 347942.7, 347942.7, 347942.7,
       347942.7, 347942.7, 347942.7, 347942.7, 347942.7, 347942.7,
       347942.7, 347942.7, 347942.7, 347942.7, 347942.7, 347942.7,
       347942.7, 347942.7, 347953.7, 347953.7, 347953.7, 347953.7,
       347953.7, 347953.7, 347953.7, 347953.7, 347953.7, 347953.7,
       347953.7, 347953.7, 347953.7, 347953.7, 347953.7, 347953.7,
       347953.7, 347953.7, 347953.7, 347953.7, 347953.7, 347953.7,
       347953.7, 347953.7, 347953.7, 347953.7, 347953.7, 34795

In [406]:
myrecord[0:100,1]

array([7339632.5, 7339632.5, 7339632.5, 7339632.5, 7339632.5, 7339632.5,
       7339632.5, 7339632.5, 7339632.5, 7339632.5, 7339632.5, 7339632.5,
       7339632.5, 7339632.5, 7339632.5, 7339632.5, 7339632.5, 7339632.5,
       7339632.5, 7339632.5, 7339632.5, 7339632.5, 7339632.5, 7339632.5,
       7339632.5, 7339632.5, 7339632.5, 7339632.5, 7339632.5, 7339632.5,
       7339632.5, 7339640.7, 7339640.7, 7339640.7, 7339640.7, 7339640.7,
       7339640.7, 7339640.7, 7339640.7, 7339640.7, 7339640.7, 7339640.7,
       7339640.7, 7339640.7, 7339640.7, 7339640.7, 7339640.7, 7339640.7,
       7339640.7, 7339640.7, 7339640.7, 7339640.7, 7339640.7, 7339640.7,
       7339640.7, 7339640.7, 7339640.7, 7339640.7, 7339640.7, 7339640.7,
       7339640.7, 7339640.7, 7339648.8, 7339648.8, 7339648.8, 7339648.8,
       7339648.8, 7339648.8, 7339648.8, 7339648.8, 7339648.8, 7339648.8,
       7339648.8, 7339648.8, 7339648.8, 7339648.8, 7339648.8, 7339648.8,
       7339648.8, 7339648.8, 7339648.8, 7339648.8, 

In [407]:
myrecord[0:100,2]

array([583.58 , 581.58 , 577.38 , 572.76 , 567.68 , 562.09 , 555.94 ,
       549.175, 541.735, 533.555, 524.555, 514.655, 503.765, 491.785,
       478.605, 464.105, 448.155, 430.61 , 411.31 , 390.08 , 366.73 ,
       341.045, 312.79 , 281.71 , 247.52 , 209.91 , 168.54 , 123.035,
        72.98 ,  17.92 , -39.76 , 583.63 , 581.63 , 577.43 , 572.81 ,
       567.73 , 562.14 , 555.99 , 549.225, 541.785, 533.605, 524.605,
       514.705, 503.815, 491.835, 478.655, 464.155, 448.205, 430.66 ,
       411.36 , 390.13 , 366.78 , 341.095, 312.84 , 281.76 , 247.57 ,
       209.96 , 168.59 , 123.085,  73.03 ,  17.97 , -39.71 , 583.97 ,
       581.97 , 577.77 , 573.15 , 568.07 , 562.48 , 556.33 , 549.565,
       542.125, 533.945, 524.945, 515.045, 504.155, 492.175, 478.995,
       464.495, 448.545, 431.   , 411.7  , 390.47 , 367.12 , 341.435,
       313.18 , 282.1  , 247.91 , 210.3  , 168.93 , 123.425,  73.37 ,
        18.31 , -39.37 , 584.29 , 582.29 , 578.09 , 573.47 , 568.39 ,
       562.8  , 556.

In [5]:
install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py

SyntaxError: invalid syntax (<ipython-input-5-054c82024b60>, line 1)

In [6]:
myshaperecord[:3]

[<shapefile._ShapeRecord instance at 0x7f28f34d6290>,
 <shapefile._ShapeRecord instance at 0x7f28f34d62d8>,
 <shapefile._ShapeRecord instance at 0x7f28f34d6320>]

In [7]:
x=[y.record for y in myshaperecord if y.record[-2]>0]

In [10]:
myshaperecord[1].record

[0,
 '160287',
 0.000862,
 0.0019,
 0.00114,
 0.00176,
 0.000862,
 4.0,
 3.37731,
 0.00566225,
 347931.8,
 7339632.5,
 581.58,
 0,
 1]

In [13]:
x[332:335]

[[0,
  '164927',
  5.32e-05,
  5.45e-05,
  6.63e-05,
  9.3,
  5.32e-05,
  11.41,
  1.07123,
  0.0,
  351917.1,
  7389543.0,
  889.335,
  1,
  2],
 [0,
  '164927',
  5.88e-05,
  6.47e-05,
  7.18e-05,
  0.426,
  5.88e-05,
  12.55,
  1.06052,
  0.0,
  351917.1,
  7389543.0,
  877.355,
  1,
  2],
 [0,
  '164927',
  6.85e-05,
  7.89e-05,
  8.25e-05,
  0.105,
  6.85e-05,
  13.81,
  1.04991,
  0.0,
  351917.1,
  7389543.0,
  864.175,
  1,
  2]]